# First case with SHARPy

This notebook explains how to create a simple airfoil in SHARPy.

It guides the reader though the steps to create a first case in SHARPy with the minimum  complexity. It requires around an hour to complete it.

## Generation of the case

This section explains how to generate the input files for SHARPy which describe the case including the structure, the aerodynamics and the simulation details.

In [18]:
# Loading of the used packages
import numpy as np # basic mathematical and array functions
import os # Functions related to the operating system
import matplotlib.pyplot as plt # Plotting library

import sharpy.sharpy_main # Run SHARPy inside jupyter notebooks
import sharpy.utils.plotutils as pu # Plotting utilities
from sharpy.utils.constants import deg2rad # Constant to conver degrees to radians

import sharpy.utils.generate_cases as gc

The "generate_cases" module of SHARPy tries to facilitate the creation of simple cases using common parameters as inputs and hidding the tedious work. 

Once the reader has understood the big picture on how to generate a case through this notebook, we recommend to navigate inside the functions and to run the rest of the example notebooks that have a higher complexity.

### Define the case parameters

In [19]:
%config InlineBackend.figure_format = 'svg'
from IPython.display import Image
url = ('https://raw.githubusercontent.com/ImperialCollegeLondon/sharpy/dev_doc/docs/' + 
       'source/content/example_notebooks/images/simple_airfoil.png')
Image(url=url, width=800)

SHARPy uses SI units for its variables unless stated otherwise

In [2]:
# Geometry
chord = 1. # Chord of the airfoil
aspect_ratio = 16. # Ratio between lenght and chord
wake_length = 50 # Length of the wake in chord lengths

# Discretization
num_node = 21 # Number of nodes in the structural discretisation
num_chord_panels = 4 # Number of aerodynamic panels in the chordwise direction
num_points_camber = 200 # Number of points that will define the camber line of the airfoil

aoa_ini_deg = 2. # Angle of attack at the beginning of the simulation 
aoa_end_deg = 1. # Angle of attack at the end of the simulation 

# Operation
WSP = 25 # Wind speed 
air_density = 0.1 # Air density 

# Stiffness
mass_per_unit_length = 0.75 # Mass per unit length
mass_iner_x = 0.1           # Mass inertia around the local x axis
mass_iner_y = 0.05          # Mass inertia around the localy axis
mass_iner_z = 0.05          # Mass inertia around the localz axis
pos_cg_B = np.zeros((3))    # position of the centre of mass with respect to the elastic axis
EA = 1e7                    # Axial stiffness
GAy = 1e6                   # Shear stiffness in the local y axis
GAz = 1e6                   # Shear stiffness in the local y axis
GJ = 1e4                    # Torsional stiffness
EIy = 2e4                   # Bending stiffness in the flapwise direction
EIz = 5e6                   # Bending stiffness in the flapwise direction

# Time discretization
end_time = 10.0                 # End time of the simulation
dt = chord/num_chord_panels/WSP # Always keep one timestep per panel

In principle, we want to keep the size of the panels equal to the distance covered by the flow in one time step. 

### Generate SHARPy required inputs

The following cells will create an AeroelasticInformation class. This class is not directly used by SHARPy, it has been thought as an intermediate step between common engineering inputs and the input information that SHARPy needs. 

It has a lot of functionalities (rotate objects, assembly simple strucures ...) which can be looked up in the [documentation](https://ic-sharpy.readthedocs.io/en/master/includes/utils/generate_cases/index.html):

In [20]:
airfoil = gc.AeroelasticInformation()

Once we have initialised the class in the generate_cases module, we can check the attibutes that have to be defined (not all of them are compulsory). Let's show the structural properties that we need to define

In [21]:
airfoil.__dict__.keys()
airfoil.StructuralInformation.__dict__.keys()

dict_keys(['num_node_elem', 'num_node', 'num_elem', 'coordinates', 'connectivities', 'elem_stiffness', 'stiffness_db', 'elem_mass', 'mass_db', 'frame_of_reference_delta', 'structural_twist', 'boundary_conditions', 'beam_number', 'body_number', 'app_forces', 'lumped_mass_nodes', 'lumped_mass', 'lumped_mass_inertia', 'lumped_mass_position'])

For example, the connectivities between the nodes required by the finite element solver are still empty:

In [22]:
print(airfoil.StructuralInformation.connectivities)

None


We can also list the methods that allow us to modify the structure. Check the [documentation ](https://ic-sharpy.readthedocs.io/en/master/includes/utils/generate_cases/index.html) for further descriptions:

In [24]:
list_of_methdos = gc.list_methods(airfoil.StructuralInformation)

assembly_structures
check_StructuralInformation
compute_basic_num_elem
compute_basic_num_node
copy
create_frame_of_reference_delta
create_mass_db_from_vector
create_simple_connectivities
create_stiff_db_from_vector
generate_1to1_from_vectors
generate_fem_file
generate_full_structure
generate_uniform_beam
generate_uniform_sym_beam
rotate_around_origin
set_to_zero


In [26]:
# Define the number of nodes and the number of nodes per element
airfoil.StructuralInformation.num_node = num_node
airfoil.StructuralInformation.num_node_elem = 3
# Compute the number of elements assuming basic connections
airfoil.StructuralInformation.compute_basic_num_elem()

In [27]:
# Generate an array with the location of the nodes
node_r = np.zeros((num_node, 3))
node_r[:,1] = np.linspace(0, chord*aspect_ratio, num_node)

The following function creates a unifrom beam from the previous parameters

In [31]:
airfoil.StructuralInformation.generate_uniform_beam(node_r,
                    mass_per_unit_length,
                    mass_iner_x,
                    mass_iner_y,
                    mass_iner_z,
                    pos_cg_B,
                    EA,
                    GAy,
                    GAz,
                    GJ,
                    EIy,
                    EIz,
                    num_node_elem = airfoil.StructuralInformation.num_node_elem,
                    y_BFoR = 'x_AFoR',
                    num_lumped_mass=0)

If we now show the connectivities between the nodes, the function has created them for us. Check the documentation for further information about how the connectivities are defined in SHARPy. https://ic-sharpy.readthedocs.io/en/master/content/casefiles.html?highlight=connectivities#fem-file

In [32]:
print(airfoil.StructuralInformation.connectivities)

[[ 0  2  1]
 [ 2  4  3]
 [ 4  6  5]
 [ 6  8  7]
 [ 8 10  9]
 [10 12 11]
 [12 14 13]
 [14 16 15]
 [16 18 17]
 [18 20 19]]


Let's define the boundary conditions as clamped for node 0 and free and for the last node (-1)

In [33]:
airfoil.StructuralInformation.boundary_conditions[0] = 1
airfoil.StructuralInformation.boundary_conditions[-1] = -1

In [34]:
# Define the coordinates of the camber line of the airfoils used
# 
airfoil_camber = np.zeros((1, num_points_camber, 2))
airfoil_camber[0, :, 0] = np.linspace(0, 1, num_points_camber)

# Compute the number of panels in the wake (streamwise direction) based on the previous paramete
mstar = int(wake_length*chord/dt)

In [35]:
# Generate blade aerodynamics
airfoil.AerodynamicInformation.create_one_uniform_aerodynamics(airfoil.StructuralInformation,
                                 chord = chord,
                                 twist = 0.,
                                 sweep = 0.,
                                 num_chord_panels = num_chord_panels,
                                 m_distribution = 'uniform',
                                 elastic_axis = 0.5,
                                 num_points_camber = num_points_camber,
                                 airfoil = airfoil_camber)

let's run a first simulation with SHARPy. It will perform no computation, it will just load the data so we can plot the system we have just created.

In [40]:
# Define the simulation
SimInfo = gc.SimulationInformation()
SimInfo.set_default_values()

SimInfo.solvers['SHARPy']['flow'] = ['BeamLoader',
                        'AerogridLoader']

SimInfo.solvers['SHARPy']['case'] = 'plot'
SimInfo.solvers['SHARPy']['route'] = './'
SimInfo.solvers['SHARPy']['write_screen'] = 'off'

SimInfo.solvers['AerogridLoader']['unsteady'] = 'on'
SimInfo.solvers['AerogridLoader']['mstar'] = mstar
SimInfo.solvers['AerogridLoader']['freestream_dir'] = np.array([1.,0.,0.])
SimInfo.solvers['AerogridLoader']['wake_shape_generator'] = 'StraightWake'
SimInfo.solvers['AerogridLoader']['wake_shape_generator_input'] = {'u_inf': WSP,
                                                                   'u_inf_direction' : np.array([np.cos(aoa_ini_deg*deg2rad), 0., np.sin(aoa_ini_deg*deg2rad)]),
                                                                   'dt': dt}

gc.clean_test_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
airfoil.generate_h5_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
SimInfo.generate_solver_file()

In [42]:
# The following line of code runs SHARPy inside the jupyter notebook
# It is equivalent to run in a terminal: sharpy plot.sharpy
sharpy_output = sharpy.sharpy_main.main(['',
                                         SimInfo.solvers['SHARPy']['route'] +
                                         SimInfo.solvers['SHARPy']['case'] +
                                         '.sharpy'])
# We have hidden the output because it is useless in this simulation

In [ ]:
# The cell below just installs plotly because it is not included in SHARPy environment

In [43]:
%%capture
pip install "notebook>=5.3" "ipywidgets>=7.2"

In [44]:
# Let's plot the case we have created
# The function below just provide a quick way of generating an interactive plot
# but it is not suitable for large cases
# but luckily SHARPy has other more complex plotting capabilities
pu.plot_timestep(sharpy_output, tstep=-1, minus_mstar=(mstar - 6), plotly=True)

Next, we are going to run a static simulation of the previous system. We have already defined the required inputs for "BeamLoader" and "AerogridLoader" but we need to define the static solvers

In [45]:
# Define the simulation
SimInfo.solvers['SHARPy']['flow'] = ['BeamLoader',
                        'AerogridLoader',
                        'StaticCoupled']

SimInfo.set_variable_all_dicts('rho', air_density)

SimInfo.solvers['SHARPy']['case'] = 'static'
SimInfo.solvers['SHARPy']['write_screen'] = 'on'

SimInfo.solvers['NonLinearStatic']['print_info'] = True
SimInfo.solvers['NonLinearStatic']['max_iterations'] = 100
SimInfo.solvers['NonLinearStatic']['num_load_steps'] = 1
SimInfo.solvers['NonLinearStatic']['delta_curved'] = 0.01
SimInfo.solvers['NonLinearStatic']['min_delta'] = 1e-05
SimInfo.solvers['NonLinearStatic']['newmark_damp'] = 0.
SimInfo.solvers['NonLinearStatic']['gravity_on'] = False
SimInfo.solvers['NonLinearStatic']['gravity'] = 9.81
SimInfo.solvers['NonLinearStatic']['relaxation_factor'] = 0.

SimInfo.solvers['StaticUvlm']['horseshoe'] = True
SimInfo.solvers['StaticUvlm']['num_cores'] = 2
SimInfo.solvers['StaticUvlm']['n_rollup'] = 0
SimInfo.solvers['StaticUvlm']['rollup_dt'] = dt
SimInfo.solvers['StaticUvlm']['velocity_field_generator'] = 'SteadyVelocityField'
SimInfo.solvers['StaticUvlm']['velocity_field_input'] = {'u_inf' : WSP,
                                                         'u_inf_direction' : np.array([np.cos(aoa_ini_deg*deg2rad), 0., np.sin(aoa_ini_deg*deg2rad)])}

SimInfo.solvers['StaticCoupled']['structural_solver'] = 'NonLinearStatic'
SimInfo.solvers['StaticCoupled']['structural_solver_settings'] = SimInfo.solvers['NonLinearStatic']
SimInfo.solvers['StaticCoupled']['aero_solver'] = 'StaticUvlm'
SimInfo.solvers['StaticCoupled']['aero_solver_settings'] = SimInfo.solvers['StaticUvlm']
SimInfo.solvers['StaticCoupled']['max_iter'] = 100
SimInfo.solvers['StaticCoupled']['n_load_steps'] = 0
SimInfo.solvers['StaticCoupled']['tolerance'] = 1e-08
SimInfo.solvers['StaticCoupled']['relaxation_factor'] = 0.0

gc.clean_test_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
airfoil.generate_h5_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
SimInfo.generate_solver_file()


In [46]:
sharpy_output = sharpy.sharpy_main.main(['',
                                         SimInfo.solvers['SHARPy']['route'] +
                                         SimInfo.solvers['SHARPy']['case'] +
                                         '.sharpy'])

--------------------------------------------------------------------------------
            ######  ##     ##    ###    ########  ########  ##    ##
           ##    ## ##     ##   ## ##   ##     ## ##     ##  ##  ##
           ##       ##     ##  ##   ##  ##     ## ##     ##   ####
            ######  ######### ##     ## ########  ########     ##
                 ## ##     ## ######### ##   ##   ##           ##
           ##    ## ##     ## ##     ## ##    ##  ##           ##
            ######  ##     ## ##     ## ##     ## ##           ##
--------------------------------------------------------------------------------
Aeroelastics Lab, Aeronautics Department.
    Copyright (c), Imperial College London.
    All rights reserved.
    License available at https://github.com/imperialcollegelondon/sharpy
Running SHARPy from /home/arturo/code/sharpy/docs/source/content/example_notebooks
SHARPy being run is in /home/arturo/code/sharpy
The branch being run is dev_example
The version and com

In [47]:
pu.plot_timestep(sharpy_output, tstep=-1, minus_mstar=(mstar-6), plotly=True)

Finally, we are going to run a dynamic simulation after the previous static one.  

In [49]:
# Compute the number of time steps needed based on the previous parameters
time_steps = int(end_time/dt)

SimInfo.solvers['SHARPy']['flow'] = ['BeamLoader',
                        'AerogridLoader',
                        'StaticCoupled',
                        'AerogridPlot',
                        'BeamPlot',
                        'DynamicCoupled']

SimInfo.solvers['SHARPy']['route'] = './'
SimInfo.solvers['SHARPy']['case'] = 'dynamic'



In [50]:
SimInfo.set_variable_all_dicts('dt', dt)
SimInfo.define_num_steps(time_steps)

NameError: name 'dt' is not defined

In [51]:
SimInfo.solvers['StepUvlm']['convection_scheme'] = 0
SimInfo.solvers['StepUvlm']['num_cores'] = 8
SimInfo.solvers['StepUvlm']['velocity_field_input'] = {'u_inf' : WSP,
                                                       'u_inf_direction' : np.array([np.cos(aoa_ini_deg*deg2rad), 0., np.sin(aoa_ini_deg*deg2rad)])}

SimInfo.solvers['WriteVariablesTime']['structure_variables'] = ['pos', 'psi']
SimInfo.solvers['WriteVariablesTime']['structure_nodes'] = list(range(0, num_node))

SimInfo.solvers['NonLinearDynamicPrescribedStep']['max_iterations'] = 100
SimInfo.solvers['NonLinearDynamicPrescribedStep']['num_load_steps'] = 1
SimInfo.solvers['NonLinearDynamicPrescribedStep']['delta_curved'] = 0.01
SimInfo.solvers['NonLinearDynamicPrescribedStep']['min_delta'] = 1e-05
SimInfo.solvers['NonLinearDynamicPrescribedStep']['newmark_damp'] = 0.
SimInfo.solvers['NonLinearDynamicPrescribedStep']['gravity_on'] = False
SimInfo.solvers['NonLinearDynamicPrescribedStep']['gravity'] = 9.81
SimInfo.solvers['NonLinearDynamicPrescribedStep']['relaxation_factor'] = 0.

SimInfo.solvers['DynamicCoupled']['structural_solver'] = 'NonLinearDynamicPrescribedStep'
SimInfo.solvers['DynamicCoupled']['structural_solver_settings'] = SimInfo.solvers['NonLinearDynamicPrescribedStep']
SimInfo.solvers['DynamicCoupled']['aero_solver'] = 'StepUvlm'
SimInfo.solvers['DynamicCoupled']['aero_solver_settings'] = SimInfo.solvers['StepUvlm']
SimInfo.solvers['DynamicCoupled']['postprocessors'] = ['BeamPlot', 'AerogridPlot', 'Cleanup', 'WriteVariablesTime']
SimInfo.solvers['DynamicCoupled']['postprocessors_settings'] = {'BeamPlot': SimInfo.solvers['BeamPlot'],
                                                             'AerogridPlot': SimInfo.solvers['AerogridPlot'],
                                                             'WriteVariablesTime': SimInfo.solvers['WriteVariablesTime'],
                                                             'Cleanup': SimInfo.solvers['Cleanup']}
SimInfo.solvers['DynamicCoupled']['minimum_steps'] = 0
SimInfo.solvers['DynamicCoupled']['include_unsteady_force_contribution'] = True
SimInfo.solvers['DynamicCoupled']['relaxation_factor'] = 0.
SimInfo.solvers['DynamicCoupled']['final_relaxation_factor'] = 0.
SimInfo.solvers['DynamicCoupled']['dynamic_relaxation'] = False
SimInfo.solvers['DynamicCoupled']['relaxation_steps'] = 0
SimInfo.solvers['DynamicCoupled']['fsi_tolerance'] = 1e-8
SimInfo.solvers['DynamicCoupled']['fsi_substeps'] = 70
SimInfo.solvers['DynamicCoupled']['structural_substeps'] = 0
SimInfo.solvers['DynamicCoupled']['steps_without_unsteady_force'] = 0
SimInfo.solvers['DynamicCoupled']['pseudosteps_ramp_unsteady_force'] = 0


In [52]:
SimInfo.with_forced_vel = True
SimInfo.for_vel = np.zeros((time_steps,6), dtype=float)
SimInfo.for_acc = np.zeros((time_steps,6), dtype=float)
SimInfo.with_dynamic_forces = True
SimInfo.dynamic_forces = np.zeros((time_steps,airfoil.StructuralInformation.num_node,6), dtype=float)

In [53]:
gc.clean_test_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
airfoil.generate_h5_files(SimInfo.solvers['SHARPy']['route'], SimInfo.solvers['SHARPy']['case'])
SimInfo.generate_solver_file()
SimInfo.generate_dyn_file(time_steps)

In [54]:
sharpy_output = sharpy.sharpy_main.main(['', SimInfo.solvers['SHARPy']['route'] + SimInfo.solvers['SHARPy']['case'] + '.sharpy'])

--------------------------------------------------------------------------------
            ######  ##     ##    ###    ########  ########  ##    ##
           ##    ## ##     ##   ## ##   ##     ## ##     ##  ##  ##
           ##       ##     ##  ##   ##  ##     ## ##     ##   ####
            ######  ######### ##     ## ########  ########     ##
                 ## ##     ## ######### ##   ##   ##           ##
           ##    ## ##     ## ##     ## ##    ##  ##           ##
            ######  ##     ## ##     ## ##     ## ##           ##
--------------------------------------------------------------------------------
Aeroelastics Lab, Aeronautics Department.
    Copyright (c), Imperial College London.
    All rights reserved.
    License available at https://github.com/imperialcollegelondon/sharpy
Running SHARPy from /home/arturo/code/sharpy/docs/source/content/example_notebooks
SHARPy being run is in /home/arturo/code/sharpy
The branch being run is dev_example
The version and com

KeyboardInterrupt: 

In [ ]:
# Generate a simple plot of the timestep 

In [55]:
pu.plot_timestep(sharpy_output, tstep=-1, minus_mstar=(mstar-6), plotly=True)

In [ ]:
pu.plot_timestep(data, tstep=-1)

fig, list_plots = plt.subplots(1, 2, figsize=(12, 3))

list_plots[0].grid()
list_plots[0].set_xlabel("r/R [-]")
list_plots[0].set_ylabel("CN/d(r/R) [-]")
list_plots[0].plot(r[0]/R, CN_drR[0], '-', label='SHARPy')
list_plots[0].plot(of_rR, of_cNdrR, '-', label='OpenFAST')
list_plots[0].legend()

list_plots[1].grid()
list_plots[1].set_xlabel("r/R [-]")
list_plots[1].set_ylabel("CT/d(r/R) [-]")
list_plots[1].plot(r[0]/R, CTan_drR[0], '-', label='SHARPy')
list_plots[1].plot(of_rR, of_cTdrR, '-', label='OpenFAST')
list_plots[1].legend()

plt.show()